## Llibreria ML de PySpark
Part 1. Pràctica
Descarrega: https://archive.ics.uci.edu/ml/datasets/covertype


1. Carrega l’arxiu a un spark dataframe directament i carrega l’arxiu passant per un pas intermig de crear un format parquet i després el spark dataframe. Són els 2 dataframes iguals? Sí


In [25]:
import findspark
from pyspark.sql import SparkSession

# Inicialitzem spark
findspark.init()

# Creem una sessió de spark
spark = SparkSession.builder.appName('pysparkml').getOrCreate()
sc = spark.sparkContext

# Afegim les capçaleres del dataset
header = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']

# Carreguem el dataset a un spark dataframe amb les capçaleres
covertype_df = spark.read.csv('covtype.data', header=False, inferSchema=True)
covertype_df = covertype_df.toDF(*header)

# Convertim a parquet
#covertype_df.write.parquet('covertype.parquet', mode='overwrite')
#covertype_parquet_df = spark.read.parquet()

# Comprovem que els dos dataframes són iguals
#covertype_df.count() == covertype_parquet_df.count()

2. Elimina un dels 2 dataframe i realitza un EDA.

In [26]:
# Eliminem el dataframe que no utilitzarem
#covertype_parquet_df.unpersist()

# Realitzem un EDA general del dataframe
covertype_df.printSchema()
covertype_df.describe().show()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area1: integer (nullable = true)
 |-- Wilderness_Area2: integer (nullable = true)
 |-- Wilderness_Area3: integer (nullable = true)
 |-- Wilderness_Area4: integer (nullable = true)
 |-- Soil_Type1: integer (nullable = true)
 |-- Soil_Type2: integer (nullable = true)
 |-- Soil_Type3: integer (nullable = true)
 |-- Soil_Type4: integer (nullable = true)
 |-- Soil_Type5: integer (nullable = true)
 |-- Soil_Type6: integer (nullable = true)
 |-- Soil_Type7: integer

3. Elimina duplicats si n’hi ha. Busca outliers i, si n’hi ha, tracta’ls.

In [27]:
# Agafem un sample del dataframe per a fer un EDA més rapid
covertype_df = covertype_df.sample(False, 0.1, seed=0)

# Eliminem duplicats
covertype_df = covertype_df.dropDuplicates()

# Comprovem que no hi ha duplicats
if covertype_df.count() == covertype_df.dropDuplicates().count():
    print('No hi ha duplicats')
else:
    print('Hi ha duplicats')

# Eliminem nuls
covertype_df = covertype_df.na.drop()

# Nivellem la quantitat de Cover_Type per a que no hi hagi desequilibri en el model.
covertype_df.groupBy('Cover_Type').count().show()
covertype_df = covertype_df.sampleBy('Cover_Type', fractions={1: 0.1, 2: 0.1, 3: 0.5, 4: 1, 5: 1, 6: 1, 7: 1}, seed=0)
covertype_df.groupBy('Cover_Type').count().show()


No hi ha duplicats
+----------+-----+
|Cover_Type|count|
+----------+-----+
|         1|20920|
|         6| 1786|
|         3| 3536|
|         5|  934|
|         4|  268|
|         7| 2004|
|         2|28242|
+----------+-----+

+----------+-----+
|Cover_Type|count|
+----------+-----+
|         1| 2228|
|         6| 1786|
|         3| 1725|
|         5|  934|
|         4|  268|
|         7| 2004|
|         2| 2918|
+----------+-----+



In [28]:
# Tractador d'outliers
def treat_outliers(df, col, search):
    # Calculem el quartil inferior
    q1 = df.approxQuantile(col, [0.25], 0.0)[0]
    # Calculem el quartil superior
    q3 = df.approxQuantile(col, [0.75], 0.0)[0]
    # Calculem el rang interquartil
    iqr = q3 - q1
    # Calculem el límit inferior
    lower_bound = q1 - (1.5 * iqr)
    # Calculem el límit superior
    upper_bound = q3 + (1.5 * iqr)
    # Retornem el dataframe sense outliers
    if search:
        return df.filter((df[col] < lower_bound) | (df[col] > upper_bound))
    else:
        return df.filter((df[col] >= lower_bound) & (df[col] <= upper_bound))

for col in covertype_df.columns:
    # Tractem els outliers de cada columna
    print("Tractant:", col)
    covertype_df = treat_outliers(covertype_df, col, False)
    # Comprovem que no hi ha outliers
    print("Outliers restants: ",treat_outliers(covertype_df, col, True).count())

Tractant: Elevation
Outliers restants:  0
Tractant: Aspect
Outliers restants:  0
Tractant: Slope
Outliers restants:  0
Tractant: Horizontal_Distance_To_Hydrology
Outliers restants:  115
Tractant: Vertical_Distance_To_Hydrology
Outliers restants:  121
Tractant: Horizontal_Distance_To_Roadways
Outliers restants:  110
Tractant: Hillshade_9am
Outliers restants:  76
Tractant: Hillshade_Noon
Outliers restants:  120
Tractant: Hillshade_3pm
Outliers restants:  0
Tractant: Horizontal_Distance_To_Fire_Points
Outliers restants:  101
Tractant: Wilderness_Area1
Outliers restants:  0
Tractant: Wilderness_Area2
Outliers restants:  0
Tractant: Wilderness_Area3
Outliers restants:  0
Tractant: Wilderness_Area4
Outliers restants:  0
Tractant: Soil_Type1
Outliers restants:  0
Tractant: Soil_Type2
Outliers restants:  0
Tractant: Soil_Type3
Outliers restants:  0
Tractant: Soil_Type4
Outliers restants:  0
Tractant: Soil_Type5
Outliers restants:  0
Tractant: Soil_Type6
Outliers restants:  0
Tractant: Soil_Typ

4. Problema de classificació de la variable COVER TYPE amb algoritme Regressió Logística del modul ML de PySpark.
    Fixa’t en com es fa per agrupar i manipular les features a Spark.
    Fixa’t en la transformació VectorAssembler.
    Fixa’t amb la transformació Pipeline.

In [29]:
import pyspark.ml.classification as cl
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Creem un vector de features
feature_cols = covertype_df.columns[:-1]

# Creem un objecte de la classe VectorAssembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# Creem un objecte de la classe LogisticRegression
lr = cl.LogisticRegression(featuresCol='features', labelCol='Cover_Type', maxIter=10)

# Creem un objecte de la classe Pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Dividim el dataset en train i test
train, test = covertype_df.randomSplit([0.7, 0.3], seed=1234)

# Entrenem el model
model = pipeline.fit(train)

# Fem prediccions
predictions = model.transform(test)

# Mostrem l'accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='Cover_Type', predictionCol='prediction', metricName='accuracy')
print('Accuracy: ', evaluator.evaluate(predictions))

Accuracy:  0.6142857142857143


5. Problema de classificació de la variable COVER TYPE amb algoritme Random forest amb modul ML de PySpark.

In [30]:
# Creem un objecte de la classe RandomForestClassifier
rf = cl.RandomForestClassifier(featuresCol='features', labelCol='Cover_Type', numTrees=10)

# Creem un objecte de la classe Pipeline
pipeline_rf = Pipeline(stages=[assembler, rf])

# Entrenem el model
model_rf = pipeline_rf.fit(train)

# Fem prediccions
predictions_rf = model_rf.transform(test)

# Mostrem l'accuracy
print('Accuracy: ', evaluator.evaluate(predictions_rf))

Accuracy:  0.6761904761904762


6. Comparativa de les métriques d’error.

In [31]:
from pyspark.ml.evaluation import RegressionEvaluator

# Creem els evaluadors
evaluator_mse = RegressionEvaluator(labelCol='Cover_Type', predictionCol='prediction', metricName='mse')
evaluator_rmse = RegressionEvaluator(labelCol='Cover_Type', predictionCol='prediction', metricName='rmse')
evaluator_r2 = RegressionEvaluator(labelCol='Cover_Type', predictionCol='prediction', metricName='r2')
evaluator_mae = RegressionEvaluator(labelCol='Cover_Type', predictionCol='prediction', metricName='mae')

# Comparem les métriques d'error dels dos models
print('Logistic Regression')
print('MSE: ', evaluator_mse.evaluate(predictions))
print('RMSE: ', evaluator_rmse.evaluate(predictions))
print('R2: ', evaluator_r2.evaluate(predictions))
print('MAE: ', evaluator_mae.evaluate(predictions))
print('Random Forest')
print('MSE: ', evaluator_mse.evaluate(predictions_rf))
print('RMSE: ', evaluator_rmse.evaluate(predictions_rf))
print('R2: ', evaluator_r2.evaluate(predictions_rf))
print('MAE: ', evaluator_mae.evaluate(predictions_rf))

Logistic Regression
MSE:  4.714285714285714
RMSE:  2.1712405933672376
R2:  0.30802585167489194
MAE:  1.0809523809523809
Random Forest
MSE:  3.3833333333333333
RMSE:  1.8393839548428526
R2:  0.5033862299141523
MAE:  0.8214285714285714


7. «Hyperparameters-tuning» amb modul ML.tuning de PySpark
Necessites fer: import pyspark.ml.tuning
Es proposa fer un «grid» al problema de classificació amb Regressió Logística.
Fixa’t en regParam i en elasticNetParam per afegir rangs al objecte del «grid».

In [32]:
import pyspark.ml.tuning

# Creem un objecte de la classe regParam
regParam = pyspark.ml.tuning.ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.1, 0.5]).build()

# Creem un objecte de la classe elasticNetParam
elasticNetParam = pyspark.ml.tuning.ParamGridBuilder().addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]).build()

# Creem un objecte de la classe ParamGridBuilder
grid = pyspark.ml.tuning.ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.1, 0.5]).addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]).build()

# Creem un objecte de la classe CrossValidator
crossvalidator = pyspark.ml.tuning.CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator, numFolds=3)

# Entrenem el model
model_cv = crossvalidator.fit(train)

# Fem prediccions
predictions_cv = model_cv.transform(test)

# Mostrem l'accuracy
print('Accuracy: ', evaluator.evaluate(predictions_cv))

Accuracy:  0.6190476190476191


8. Problema de regressió de la variable ELEVATION amb algoritme Random forest regressor del modul ML de PySpark

In [33]:
import pyspark.ml.regression as reg

# Creem un objecte de la classe RandomForestRegressor
rf = reg.RandomForestRegressor(featuresCol='features', labelCol='Elevation')

# Creem un objecte de la classe Pipeline
pipeline_rf = Pipeline(stages=[assembler, rf])

# Entrenem el model
model_rf = pipeline_rf.fit(train)

# Fem prediccions
predictions_rf = model_rf.transform(test)

# Mostrem l'accuracy
print('Accuracy: ', evaluator.evaluate(predictions_rf))

Accuracy:  0.0


9. Problema de regressió de la variable ELEVATION amb algoritme Gradient-boosted Tree regressor del modul ML de PySpark

In [34]:
# Creem un objecte de la classe GBTRegressor
gbt = reg.GBTRegressor(featuresCol='features', labelCol='Elevation')

# Creem un objecte de la classe Pipeline
pipeline_gbt = Pipeline(stages=[assembler, gbt])

# Entrenem el model
model_gbt = pipeline_gbt.fit(train)

# Fem prediccions
predictions_gbt = model_gbt.transform(test)

# Mostrem l'accuracy
print('Accuracy: ', evaluator.evaluate(predictions_gbt))

Accuracy:  0.0


10. Comparativa de les métriques d’error.

In [35]:
# Comparem les métriques d'error dels dos models
print('Random Forest')
print('MSE: ', evaluator_mse.evaluate(predictions_rf))
print('RMSE: ', evaluator_rmse.evaluate(predictions_rf))
print('R2: ', evaluator_r2.evaluate(predictions_rf))
print('MAE: ', evaluator_mae.evaluate(predictions_rf))
print('Gradient Boosted Trees')
print('MSE: ', evaluator_mse.evaluate(predictions_gbt))
print('RMSE: ', evaluator_rmse.evaluate(predictions_gbt))
print('R2: ', evaluator_r2.evaluate(predictions_gbt))
print('MAE: ', evaluator_mae.evaluate(predictions_gbt))

Random Forest
MSE:  9664561.638649119
RMSE:  3108.7878085596517
R2:  -1418586.504905398
MAE:  3103.0350121769266
Gradient Boosted Trees
MSE:  9686865.338480964
RMSE:  3112.372943347401
R2:  -1421860.2953862916
MAE:  3105.2195392652948


11. Clustering
import pyspark.ml.clustering
Fer un k-means de 7 centroides del mateix DataFrame del inici de la pràctica, què observes?

In [36]:
import pyspark.ml.clustering as cl

# Creem un objecte de la classe KMeans
kmeans = cl.KMeans(featuresCol='features', k=7)

# Creem un objecte de la classe Pipeline
pipeline_kmeans = Pipeline(stages=[assembler, kmeans])

# Entrenem el model
model_kmeans = pipeline_kmeans.fit(covertype_df)

# Fem prediccions
predictions_kmeans = model_kmeans.transform(covertype_df)

# Mostrem les prediccions
predictions_kmeans.select('Cover_Type', 'prediction').show()
print("Observo que el model no ha fet un bon clustering, ja que hi ha moltes observacions que no han quedat assignades al cluster que corresponen.")

+----------+----------+
|Cover_Type|prediction|
+----------+----------+
|         1|         1|
|         2|         6|
|         1|         2|
|         7|         0|
|         1|         0|
|         2|         5|
|         2|         2|
|         2|         6|
|         1|         4|
|         7|         5|
|         7|         3|
|         7|         3|
|         7|         3|
|         7|         0|
|         2|         6|
|         2|         6|
|         2|         4|
|         2|         5|
|         1|         3|
|         7|         2|
+----------+----------+
only showing top 20 rows

Observo que el model no ha fet un bon clustering, ja que hi ha moltes observacions que no han quedat assignades al cluster que corresponen.
